In [15]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# import torch

# def create_glider(grid, x, y):
#     glider = np.array([[0, 1, 0],
#                        [0, 0, 1],
#                        [1, 1, 1]])
#     grid[x:x+3, y:y+3] = glider

# def update_grid(grid):
#     N = grid.shape[0]
#     new_grid = grid.copy()
#     for i in range(N):
#         for j in range(N):
#             total = int((grid[i, (j-1)%N] + grid[i, (j+1)%N] + 
#                          grid[(i-1)%N, j] + grid[(i+1)%N, j] + 
#                          grid[(i-1)%N, (j-1)%N] + grid[(i-1)%N, (j+1)%N] + 
#                          grid[(i+1)%N, (j-1)%N] + grid[(i+1)%N, (j+1)%N]))
#             if grid[i, j] == 1:
#                 if (total < 2) or (total > 3):
#                     new_grid[i, j] = 0
#             else:
#                 if total == 3:
#                     new_grid[i, j] = 1
#     return new_grid

# def run_game_of_life(N, steps, seed):
#     np.random.seed(seed)
    
#     grid = np.zeros((N, N))
#     x, y = np.random.randint(0, N-3, 2)
#     create_glider(grid, x, y)
    
#     grids = [grid.copy()]
#     for _ in range(steps - 1):
#         grid = update_grid(grid)
#         grids.append(grid.copy())
    
#     return np.array(grids)

# def animate_game_of_life(grids):
#     fig, ax = plt.subplots()
#     img = ax.imshow(grids[0], interpolation='nearest', cmap='binary')
    
#     def update(frame):
#         img.set_array(grids[frame])
#         return [img]
    
#     ani = FuncAnimation(fig, update, frames=len(grids), interval=200, blit=True)
#     plt.close(fig)  # Prevent the static plot from displaying
#     return ani

# # Set parameters
# N = 10  # Grid size
# steps = 50  # Number of time steps
# seed = 42  # Random seed

# # Run the Game of Life simulation
# grids = run_game_of_life(N, steps, seed)

# # Create and display the animation
# ani = animate_game_of_life(grids)
# from IPython.display import HTML
# HTML(ani.to_jshtml())

# # Convert the numpy array to a PyTorch tensor
# tensor_grids = torch.from_numpy(grids).float()

# print(f"Shape of the tensor: {tensor_grids.shape}")
# print(f"Type of the tensor: {tensor_grids.dtype}")


Shape of the tensor: torch.Size([50, 10, 10])
Type of the tensor: torch.float32


In [19]:
# from IPython.display import HTML
# HTML(ani.to_jshtml())


In [17]:
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def create_glider(grid, x, y):
    glider = torch.tensor([[0, 1, 0],
                           [0, 0, 1],
                           [1, 1, 1]], dtype=torch.float32)
    grid[x:x+3, y:y+3] = glider

def update_grid(grid):
    N = grid.shape[0]
    total = (
        grid.roll(1, 0) + grid.roll(-1, 0) +
        grid.roll(1, 1) + grid.roll(-1, 1) +
        grid.roll(1, 0).roll(1, 1) + grid.roll(1, 0).roll(-1, 1) +
        grid.roll(-1, 0).roll(1, 1) + grid.roll(-1, 0).roll(-1, 1)
    )
    return ((grid == 1) & ((total == 2) | (total == 3))) | ((grid == 0) & (total == 3))

def run_game_of_life(N, steps, seed):
    torch.manual_seed(seed)
    
    grid = torch.zeros((N, N), dtype=torch.float32)
    x, y = torch.randint(0, N-3, (2,))
    create_glider(grid, x, y)
    
    grids = [grid.clone()]
    for _ in range(steps - 1):
        grid = update_grid(grid).float()
        grids.append(grid.clone())
    
    return torch.stack(grids)

def animate_game_of_life(grids):
    fig, ax = plt.subplots()
    img = ax.imshow(grids[0].cpu().numpy(), interpolation='nearest', cmap='binary')
    
    def update(frame):
        img.set_array(grids[frame].cpu().numpy())
        return [img]
    
    ani = FuncAnimation(fig, update, frames=len(grids), interval=200, blit=True)
    plt.close(fig)  # Prevent the static plot from displaying
    return ani

# Set parameters
N = 20  # Grid size
steps = 50  # Number of time steps
seed = 42  # Random seed

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Run the Game of Life simulation
grids = run_game_of_life(N, steps, seed).to(device)

# Create and display the animation
ani = animate_game_of_life(grids)
from IPython.display import HTML
HTML(ani.to_jshtml())

print(f"Shape of the tensor: {grids.shape}")
print(f"Type of the tensor: {grids.dtype}")
print(f"Device of the tensor: {grids.device}")


Using device: cuda


Shape of the tensor: torch.Size([50, 20, 20])
Type of the tensor: torch.float32
Device of the tensor: cuda:0


In [20]:
from IPython.display import HTML
HTML(ani.to_jshtml())
